FUNCION PARA EXTRAER LAS CANCIONES DEL API DE SPOTIFY ⏮️⏹️⏩

In [ ]:
"""
EXTRACTOR DE DATOS MUSICALES DE SPOTIFY
Extrae 200 canciones por género y año (2019-2022)
Total esperado: 3,200 canciones
"""

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
from datetime import datetime

# ==================== CONFIGURACIÓN ====================
# 👇 PON TUS CREDENCIALES AQUÍ
CLIENT_ID = 'TU_CLIENT_ID_AQUI'
CLIENT_SECRET = 'TU_CLIENT_SECRET_AQUI'

# Parámetros del proyecto
GENEROS = ['rock', 'clásica', 'reggaetón', 'pop']
AÑOS = range(2019, 2023)  # 2019, 2020, 2021, 2022
CANCIONES_POR_BUSQUEDA = 200

# ==================== CONEXIÓN A SPOTIFY ====================
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

# Lista global para almacenar datos
datos_canciones = []


# ==================== FUNCIÓN 1: EXTRAER DATOS ====================
def extraer_datos_cancion(track, genero, año):
    """Extrae toda la información de una canción"""
    try:
        track_id = track['id']
        audio_features = sp.audio_features(track_id)[0]
        
        if audio_features:
            return {
                'id': track_id,
                'nombre': track['name'],
                'artista': track['artists'][0]['name'],
                'album': track['album']['name'],
                'genero': genero,
                'año_busqueda': año,
                'fecha_lanzamiento': track['album']['release_date'],
                'popularidad': track['popularity'],
                'duracion_ms': track['duration_ms'],
                'duracion_min': round(track['duration_ms'] / 60000, 2),
                'valence': audio_features['valence'],
                'energy': audio_features['energy'],
                'danceability': audio_features['danceability'],
                'acousticness': audio_features['acousticness'],
                'instrumentalness': audio_features['instrumentalness'],
                'liveness': audio_features['liveness'],
                'speechiness': audio_features['speechiness'],
                'tempo': audio_features['tempo'],
                'loudness': audio_features['loudness'],
            }
    except Exception as e:
        print(f"  ⚠️  Error: {track.get('name', 'desconocida')}")
        return None


# ==================== FUNCIÓN 2: BUSCAR CANCIONES ====================
def buscar_canciones(genero, año, limite=200):
    """Busca canciones de un género y año específico"""
    print(f"\n🎵 Buscando: {genero.upper()} - {año}")
    
    canciones = []
    ids_vistos = set()
    offset = 0
    
    while len(canciones) < limite:
        try:
            # Buscar en Spotify
            query = f"genre:{genero} year:{año}"
            resultados = sp.search(q=query, type='track', limit=50, offset=offset, market='ES')
            tracks = resultados['tracks']['items']
            
            if not tracks:
                break
            
            # Procesar cada track
            for track in tracks:
                if track['id'] not in ids_vistos and len(canciones) < limite:
                    ids_vistos.add(track['id'])
                    datos = extraer_datos_cancion(track, genero, año)
                    if datos:
                        canciones.append(datos)
                        
                        if len(canciones) % 20 == 0:
                            print(f"  ✓ {len(canciones)} canciones...")
            
            offset += 50
            time.sleep(0.5)  # Pausa para no saturar API
            
        except Exception as e:
            print(f"  ❌ Error en búsqueda: {e}")
            break
    
    print(f"  ✅ Total: {len(canciones)} canciones")
    return canciones


# ==================== FUNCIÓN 3: PROCESO PRINCIPAL ====================
def main():
    """Ejecuta todo el proceso de extracción"""
    print("\n" + "="*70)
    print("🎼 EXTRACTOR DE DATOS MUSICALES DE SPOTIFY")
    print("="*70)
    print(f"Géneros: {', '.join(GENEROS)}")
    print(f"Años: {list(AÑOS)}")
    print(f"Objetivo: {CANCIONES_POR_BUSQUEDA} canciones x género x año")
    print(f"Total esperado: {len(GENEROS) * len(AÑOS) * CANCIONES_POR_BUSQUEDA} canciones")
    print("="*70)
    
    inicio = time.time()
    
    # Extraer canciones por género y año
    for genero in GENEROS:
        for año in AÑOS:
            canciones = buscar_canciones(genero, año, CANCIONES_POR_BUSQUEDA)
            datos_canciones.extend(canciones)
    
    tiempo_total = time.time() - inicio
    
    # Crear DataFrame
    df = pd.DataFrame(datos_canciones)
    
    # Mostrar resumen
    print("\n" + "="*70)
    print("📊 RESUMEN FINAL")
    print("="*70)
    print(f"✅ Total extraído: {len(datos_canciones)} canciones")
    print(f"⏱️  Tiempo: {tiempo_total/60:.1f} minutos")
    
    print("\n📈 Por género:")
    print(df['genero'].value_counts().to_string())
    
    print("\n📅 Por año:")
    print(df['año_busqueda'].value_counts().sort_index().to_string())
    
    # Guardar en CSV
    nombre_archivo = f'spotify_data_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
    print(f"\n💾 Guardado en: {nombre_archivo}")
    
    # Vista previa
    print("\n🔍 Primeras 5 canciones:")
    print(df[['nombre', 'artista', 'genero', 'popularidad', 'valence']].head().to_string())
    
    print("\n" + "="*70)
    print("✅ PROCESO COMPLETADO")
    print("="*70)
    
    return df


# ==================== EJECUTAR ====================
if __name__ == "__main__":
    try:
        df_final = main()
        
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrumpido por el usuario")
        if datos_canciones:
            df = pd.DataFrame(datos_canciones)
            df.to_csv('spotify_data_PARCIAL.csv', index=False, encoding='utf-8-sig')
            print(f"💾 Guardadas {len(datos_canciones)} canciones parciales")
    
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        if datos_canciones:
            df = pd.DataFrame(datos_canciones)
            df.to_csv('spotify_data_ERROR.csv', index=False, encoding='utf-8-sig')
            print(f"💾 Guardadas {len(datos_canciones)} canciones antes del error")
            